### Using just SQL

Having a quality data in the database is sufficient for any investigation. We can use any programming language we want. 
For example, it is possible to replicate Collective2 Scoring Workbench functionality using just SQL.
See the following examples.

#### SQL - Example 1

This is an SQL code for Daniil's formula above. Using some PostgreSQL UI, we can inspect all available data of the each selected strategy. 
It's an ideal form for exploring the data in details.
We can sorting any column up and down when we are trying to find extremes. We can modify formula or selection easily. We can filter results in IDE...

If we are working with a local database, we can see results immediately. 

~~~sql
#  ---------- Plain with all data  ----------
select * from (
			select
			    StrategyId, 
			    StrategyName, 
			    Added,
			    -- Risk
			    ((20 - MaxDrawdownPcnt * (20/45))
			    + case when MaxWorstLossPercentEquity060 is null then 0 else 20 - MaxWorstLossPercentEquity060 * (20.0/8) end  
			    + (10 - OptionPercent * (10/1))
			    + case when ShortOptionsCovered >= 100 then 10 else ShortOptionsCovered / 100 * 10 end 
			    -- Behavior
			    + case when AgeDays >= 720 then 40 else AgeDays/720 * 40 end  
			    + (20 - DailyMaxLevMax * (20/10)) 
			    + case when TradeDaysAgo <= 60 then 20 else 0 end
			    + case when PcntMonthsProfitable >= 100 then 5 else PcntMonthsProfitable / 100 * 5 end
			    + case when NumTrades >= 100 then 20 else NumTrades / 100 * 20 end
			    + case when DeltaEquity90Days <= 0 and DeltaEquity90Days > -7 then 5 else 0 end
			    -- Profitability 
			    + case when AnnReturn > 0.6 then 20 else AnnReturn / 0.6 * 20 end)
			    as Score, 
			    c2score_ScoringWorkbenchView.*
			from c2score_ScoringWorkbenchView
			where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
			) as sub
 where Score > 0
 order by Score desc;
~~~

![Image](./PostgreSQL_Daniil01_plain.png)